# Exemple de pattern Observer

On met en place une station météo qui ve mesurer température, humidité, pression.
Elle notifie différents affichages lorsque les conditions changent.

In [1]:
from __future__ import annotations
from abc import ABC, abstractmethod
from random import randrange
from typing import List

### Classe Sujet

In [2]:
class Subject(ABC):
    """
    The Subject interface declares a set of methods for managing subscribers.
    """

    @abstractmethod
    def registerObserver(self, observer: Observer) -> None:
        """
        Attach an observer to the subject.
        """
        pass

    @abstractmethod
    def removeObserver(self, observer: Observer) -> None:
        """
        Detach an observer from the subject.
        """
        pass

    @abstractmethod
    def notifyObservers(self) -> None:
        """
        Notify all observers about an event.
        """
        pass

### Classe sujet concrete

In [3]:
class WeatherData(Subject):
    """
    The Subject owns some important state and notifies observers when the state
    changes.
    """

    _state: int = None
    """
    For the sake of simplicity, the Subject's state, essential to all
    subscribers, is stored in this variable.
    """

    _observers: List[Observer] = []
    """
    List of subscribers. In real life, the list of subscribers can be stored
    more comprehensively (categorized by event type, etc.).
    """
    
    _temperature: Float = None
    _humidity: Float = None
    _pressure: FLoat = None
    

    def registerObserver(self, observer: Observer) -> None:
        print("Subject: Attached an observer.")
        self._observers.append(observer)

    def removeObserver(self, observer: Observer) -> None:
        self._observers.remove(observer)

    """
    The subscription management methods.
    """

    def notifyObservers(self) -> None:
        """
        Trigger an update in each subscriber.
        """
        print("Subject: Notifying observers...")
        for observer in self._observers:
            observer.update(self._temperature, self._humidity, self._pressure)
            observer.display()

    def measurementsChanged(self) -> None:
        self.notifyObservers()
    
    
    def setMeasurements(self, temperature: Float, humidity:Float, pressure:Float) -> None:
        self._temperature = temperature
        self._humidity = humidity
        self._pressure = pressure
        print("\nSubject: Changement des conditions météo.")
        print(f"T= {self._temperature} degrés, h= {self._humidity} %, p={self._pressure} hPa")
        self.measurementsChanged()

### Classe Observer

In [4]:
class Observer(ABC):
    """
    The Observer interface declares the update method, used by subjects.
    """

    @abstractmethod
    def update(self, temperature: Float, humidity: Float, pressure: Float) -> None:
        """
        Receive update from subject.
        """
        pass


    """
    Concrete Observers react to the updates issued by the Subject they had been
    attached to.
    """

### Classes Observer Concrètes
Les différents affichages de notre station météo

In [5]:
class CurrentConditionsDisplay(Observer):
    _weatherData: Subject = None
    _temperature: Float = None
    _humidity: FLoat = None
    _pressure: Float = None
    
    def __init__(self, weatherData: Subject) -> None: #Constructeur
        self._weatherData = weatherData
        self._weatherData.registerObserver(self)
    
    def update(self, temperature: Float, humidity: Float, pressure: Float) -> None:
        self._temperature = temperature
        self._humidity = humidity
        self._pressure = pressure
        
    def display(self) -> None:
        print (f"Current conditions display : T= {self._temperature}°C, H= {self._humidity}%, p= {self._pressure}hPa.")

In [6]:
class ForecastDisplay(Observer):
    _weatherData: Subject = None
    _currentPressure: Float = None
    _lastPressure: Float = None

    def __init__(self, weatherData: Subject) -> None: #Constructeur
        self._weatherData = weatherData
        self._weatherData.registerObserver(self)
        self._currentPressure = None
        self._lastPressure = None
    
    
    def update(self, temperature: Float, humidity: Float, pressure: Float) -> None:
        if self._lastPressure == None :
            self._lastPressure = pressure
        else:
             self._lastPressure = self._currentPressure
                
        self._currentPressure = pressure
    
    
    def display(self) -> None:
        print (f"Forecast Display : last Pressure = {self._lastPressure} hPa.")
        print (f"Forecast Display : Current Pressure = {self._currentPressure} hPa.")
        if self._currentPressure >= (1.05 * self._lastPressure) :
            print ("ForecastDisplay : It gets better!")
        elif self._currentPressure <= (0.95 * self._lastPressure) :
            print ("ForecastDisplay : It gets worse!")
        else:
            print ("Forecast Display : Pretty much the same!")

## Code test

In [7]:
weatherData = WeatherData()

In [8]:
currentDisplay = CurrentConditionsDisplay(weatherData)

Subject: Attached an observer.


In [9]:
weatherData.setMeasurements(23,65,1015)


Subject: Changement des conditions météo.
T= 23 degrés, h= 65 %, p=1015 hPa
Subject: Notifying observers...
Current conditions display : T= 23°C, H= 65%, p= 1015hPa.


In [10]:
currentDisplay.display()

Current conditions display : T= 23°C, H= 65%, p= 1015hPa.


In [11]:
forecastDisplay = ForecastDisplay(weatherData)

Subject: Attached an observer.


In [12]:
weatherData.setMeasurements(19,85,1005)


Subject: Changement des conditions météo.
T= 19 degrés, h= 85 %, p=1005 hPa
Subject: Notifying observers...
Current conditions display : T= 19°C, H= 85%, p= 1005hPa.
Forecast Display : last Pressure = 1005 hPa.
Forecast Display : Current Pressure = 1005 hPa.
Forecast Display : Pretty much the same!


In [13]:
weatherData.setMeasurements(19,92,950)


Subject: Changement des conditions météo.
T= 19 degrés, h= 92 %, p=950 hPa
Subject: Notifying observers...
Current conditions display : T= 19°C, H= 92%, p= 950hPa.
Forecast Display : last Pressure = 1005 hPa.
Forecast Display : Current Pressure = 950 hPa.
ForecastDisplay : It gets worse!


In [14]:
weatherData.setMeasurements(19,92,1030)


Subject: Changement des conditions météo.
T= 19 degrés, h= 92 %, p=1030 hPa
Subject: Notifying observers...
Current conditions display : T= 19°C, H= 92%, p= 1030hPa.
Forecast Display : last Pressure = 950 hPa.
Forecast Display : Current Pressure = 1030 hPa.
ForecastDisplay : It gets better!


In [15]:
weatherData.removeObserver(currentDisplay)

In [16]:
weatherData.setMeasurements(25,75,1020)


Subject: Changement des conditions météo.
T= 25 degrés, h= 75 %, p=1020 hPa
Subject: Notifying observers...
Forecast Display : last Pressure = 1030 hPa.
Forecast Display : Current Pressure = 1020 hPa.
Forecast Display : Pretty much the same!
